In [39]:
cd /content/drive/MyDrive/pytorch_practice

/content/drive/MyDrive/pytorch_practice


In [40]:
!git config --global user.email 'ryuzho@github.com'
!git config --global user.name 'ryuzho'
!git add .
!git status
!git commit -m 'pytorch - Nerual Network'
!git push

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   NeuralNet.ipynb
	modified:   transform.ipynb

[main e34e29a] pytorch - Nerual Network
 2 files changed, 2 insertions(+), 1 deletion(-)
 create mode 100644 NeuralNet.ipynb
 rewrite transform.ipynb (60%)
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 4.10 KiB | 1.02 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/ryuzho/pytorch_practice.git
   813a790..e34e29a  main -> main


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
    
  def forward(self, x):
    x = self.flatten(x)
    x = self.linear_relu_stack(x)
    return x

In [8]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [29]:
x = torch.rand(1,28,28, device=device)
logits = model(x)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
y_pred

tensor([8], device='cuda:0')

In [30]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [32]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
flat_image.shape

torch.Size([3, 784])

In [33]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())


torch.Size([3, 20])


In [34]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2296, -0.5940, -0.4476, -0.0724, -0.6270,  0.4234,  0.5476,  0.1822,
         -0.1941,  0.1011,  0.1561,  0.1914, -0.2699, -0.1013,  0.2766, -0.3049,
         -0.1194, -0.0753,  0.0136,  0.1346],
        [ 0.1613, -0.0500, -0.6286, -0.2830, -0.5780,  0.3538,  0.1333,  0.2079,
         -0.2401, -0.0091,  0.1989, -0.0569, -0.4189, -0.0305,  0.6381, -0.4985,
         -0.0113,  0.0737,  0.4300, -0.1863],
        [ 0.1403, -0.2811, -0.2840, -0.0663, -0.8683,  0.2723,  0.5683,  0.1508,
         -0.1822, -0.0685, -0.0485,  0.2849, -0.1928,  0.0723,  0.5439, -0.0924,
          0.0744,  0.2549,  0.3699,  0.4050]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4234, 0.5476, 0.1822, 0.0000,
         0.1011, 0.1561, 0.1914, 0.0000, 0.0000, 0.2766, 0.0000, 0.0000, 0.0000,
         0.0136, 0.1346],
        [0.1613, 0.0000, 0.0000, 0.0000, 0.0000, 0.3538, 0.1333, 0.2079, 0.0000,
         0.0000, 0.1989, 0.0000, 0.0000, 0.0000, 0.63

In [35]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

tensor([[ 0.2407,  0.2355, -0.1840, -0.1597,  0.0599, -0.3539, -0.2706, -0.1817,
         -0.0421, -0.1708],
        [ 0.3706,  0.2802, -0.0687, -0.1762,  0.1663, -0.3114, -0.1373, -0.2607,
         -0.1658, -0.1847],
        [ 0.1827,  0.3297, -0.1904, -0.1266,  0.2208, -0.2595, -0.1848, -0.2443,
         -0.1427, -0.0547]], grad_fn=<AddmmBackward0>)

In [37]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.1356, 0.1349, 0.0887, 0.0908, 0.1131, 0.0748, 0.0813, 0.0889, 0.1022,
         0.0898],
        [0.1481, 0.1353, 0.0955, 0.0857, 0.1208, 0.0749, 0.0891, 0.0788, 0.0866,
         0.0850],
        [0.1232, 0.1427, 0.0848, 0.0904, 0.1280, 0.0792, 0.0853, 0.0804, 0.0890,
         0.0971]], grad_fn=<SoftmaxBackward0>)

In [38]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0218, -0.0258, -0.0112,  ..., -0.0209, -0.0010, -0.0102],
        [ 0.0266,  0.0040, -0.0205,  ..., -0.0323, -0.0232, -0.0284]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0176,  0.0036], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0126,  0.0005,  0.0334,  ...,  0.0325,  0.0438,  0.0131],
        [-0.0023,  0.0148,  0.0062,  ...,  0.0367,  0.0201, -0.0430]],
       device='cuda:0', grad_fn=<Sl

## **TORCH.AUTOGRAD**

In [49]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [50]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7fb853d5db50>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7fb853d5d890>


In [51]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0442, 0.0839, 0.2227],
        [0.0442, 0.0839, 0.2227],
        [0.0442, 0.0839, 0.2227],
        [0.0442, 0.0839, 0.2227],
        [0.0442, 0.0839, 0.2227]])
tensor([0.0442, 0.0839, 0.2227])


In [52]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)


True
False


In [53]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


In [54]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
